In [44]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random as rd

# 이투스 진짜 분석

# 고려할 것

> 한 학생의 extype==1에 있는 대표등급 -> extype==2에 있는 대표등급의 관계


## 가정
1.extype==1인 시험은 모두 난이도가 같고. extype==2 에 대해서도 마찬가지다.

2.그 사람의 등급은 대표값이 결정하고, 대표값과 다른 경우는 오류가 생긴 것이다.

예)그사람이 그 과목에서 본 등급의 1. 최빈값 2. 중앙값 3. 평균 4. 최대,최소 5.백분위 점수의 평균


3.최종 결과의 등급 예측은, 수능에서의 비율이 현재 가진 데이터의 비율(extype=1 와 
)과 같다.왜냐? 2type->1type에서의 변환식은 둘의 교집합에 의해서만 변하지만, 등급을 다시 매길때,얼마나 등급이 변경 될지의 비율이 달라지기 때문에, 수능에서도 동일하다고 가정해야한다. 단, 이때 오류가 식에 의한 오류인지, 정말 수능에서의 비율에서 나오는 오차인지 알 수 없다.

# 데이터 만든 것

In [8]:
#BIG3_elidata.to_csv(r'C:\Users\74857\데이콘야구\동국대\DATA_eli.txt',header=False,index=False,sep=',')
#BIG3_elidata = pd.read_csv(r'C:\Users\74857\데이콘야구\동국대\DATA(revised).txt',engine='python')
BIG3_elidata = pd.read_csv(r'C:\Users\74857\데이콘야구\동국대\DATA_eli.txt',engine='python')

In [63]:
EX_1list=[212,213,216,217,220,221,228,229,232,233,236,237] #222,223, 238,239는 수능 데이터이다.
EX_2list=[3002,3004,3005,3008,3009,3011,3013,3014,3021,3022,3023,3024,3025,3026,3027,3028]
#시험이 구분되는 곳


stulist = set(list(BIG3_elidata[BIG3_elidata['EXAMTYPE']==1]['MEM_NO'].unique())).intersection(set(list(BIG3_elidata[BIG3_elidata['EXAMTYPE']==2]['MEM_NO'].unique())))
intersut=list(stulist)
#1과 2를 본 학생들의 멤버번호가 저장된 곳


sampledata = BIG3_elidata.copy()
sampledata_ex1=sampledata[sampledata['EXAMTYPE']==1]
sampledata_ex2=sampledata[sampledata['EXAMTYPE']==2]


SUBLIST= BIG3_elidata['SUB_CD'].unique()
collist=['MEM_NO']
for i in np.sort(SUBLIST):
    collist.append(str(i))
    
#데이터를 만드는데 필요한 정보와 도구들

In [5]:
sampledata.head()

,EXAMTYPE,MEM_NO,SUB_CD,EXAMNO,SCORE,AC_STD,AC_PCT,RATING,CHEONGSOL_YN
0,1,14402,1000,212,47,90.0,31.0,6,0
1,1,14402,2100,212,30,87.0,32.0,6,0
2,1,14402,3000,212,50,94.0,38.0,5,0
3,1,14402,4207,212,21,47.0,44.0,5,0
4,1,14402,4213,212,15,47.0,44.0,5,0


In [7]:
##여기서 데이터가 저장될 틀을 만든다

EX1_represent=pd.DataFrame(None,columns = collist)
EX1_represent['MEM_NO']= list(BIG3_elidata[BIG3_elidata['EXAMTYPE']==1]['MEM_NO'].unique())
#주의 : 시험결과 1과 2는 다른 것으로 실행되어야 한다.
EX2_represent=pd.DataFrame(None,columns = collist)
EX2_represent['MEM_NO']= list(BIG3_elidata[BIG3_elidata['EXAMTYPE']==2]['MEM_NO'].unique())


# 먼저 대표값들을 설정할 때, 교집합에 대해서만 구한다.
# 그 다음에 가장 효율적인 것을 구한 후, 모든 데이터에 대해 그렇게 한다.

## -백분위 점수의 평균

In [12]:
for mem_no in EX1_represent['MEM_NO']:
    Sumind = sampledata_ex1[sampledata_ex1['MEM_NO']==mem_no]['AC_PCT'].groupby(sampledata_ex1['SUB_CD']).mean().add_suffix('').reset_index()
    #대표값을 만들어 임시 리스트에 저장한다
    for i in range(len(EX1_represent.columns)):
        
        #한 사람이 어떤 과목을 봤을지 모르므로 다 확인해 보는게 맞다
        if EX1_represent.columns[i] in list(Sumind['SUB_CD']):
                    EX1_represent.loc[EX1_represent['MEM_NO']==mem_no,EX1_represent.columns[i]] = Sumind[EX1_represent.columns[i]==Sumind['SUB_CD']]['AC_PCT'].iloc[0] #컬럼과 뽑아낸 과목코드가 같은 것의 값을
    if mem_no==EX1_represent['MEM_NO'].loc[len(EX1_represent['MEM_NO'])//5]:
        print("20% 완료")
    elif mem_no==EX1_represent['MEM_NO'].loc[2*len(EX1_represent['MEM_NO'])//5]:
        print("40% 완료")
    elif mem_no==EX1_represent['MEM_NO'].loc[3*len(EX1_represent['MEM_NO'])//5]:
        print("60% 완료")
    elif mem_no==EX1_represent['MEM_NO'].loc[4*len(EX1_represent['MEM_NO'])//5]:
        print("80% 완료")

print("finish")

20% 완료
40% 완료
60% 완료
80% 완료
finish


In [10]:
EX1_represent['MEM_NO'].loc[2*len(EX1_represent['MEM_NO'])//5]

4534792

In [17]:
EX1_represent.head()

,MEM_NO,1000,2100,2200,3000,4102,4103,4107,4110,4111,...,5001,5002,5003,5004,5005,5006,5007,5008,5009,7000
0,14402,27,31.5,19,32.3333,NaN,NaN,NaN,NaN,22,...,39,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.3333
1,69420,8,NaN,78,11,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,77385,71,NaN,64,55,79,60,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,58
3,83920,NaN,NaN,100,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,101936,89.5,NaN,84,76.5,82,49.5,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,98


In [ ]:
for mem_no in EX2_represent['MEM_NO']:
    Sumind = sampledata_ex2[sampledata_ex2['MEM_NO']==mem_no]['AC_PCT'].groupby(sampledata_ex2['SUB_CD']).mean().add_suffix('').reset_index()
    #대표값을 만들어 임시 리스트에 저장한다
    for i in range(len(EX2_represent.columns)):
        
        #한 사람이 어떤 과목을 봤을지 모르므로 다 확인해 보는게 맞다
        if EX2_represent.columns[i] in list(Sumind['SUB_CD']):
                    EX2_represent.loc[EX2_represent['MEM_NO']==mem_no,EX2_represent.columns[i]] = Sumind[EX2_represent.columns[i]==Sumind['SUB_CD']]['AC_PCT'].iloc[0] #컬럼과 뽑아낸 과목코드가 같은 것의 값을
    
    if mem_no==EX2_represent['MEM_NO'].loc[len(EX2_represent['MEM_NO'])//5]:
        print("20% 완료")
    elif mem_no==EX2_represent['MEM_NO'].loc[2*len(EX2_represent['MEM_NO'])//5]:
        print("40% 완료")
    elif mem_no==EX2_represent['MEM_NO'].loc[3*len(EX2_represent['MEM_NO'])//5]:
        print("60% 완료")
    elif mem_no==EX2_represent['MEM_NO'].loc[4*len(EX2_represent['MEM_NO'])//5]:
        print("80% 완료")           
print("finish")

# 최후의 대표값을 설정한 후 실행 되는 곳

In [ ]:
for mem_no in EX1_represent_Ac_std['MEM_NO']:
    Sumind = sampledata_ex1[sampledata_ex1['MEM_NO']==mem_no]['AC_STD'].groupby(sampledata_ex1['SUB_CD']).mean().add_suffix('').reset_index()
    #대표값을 만들어 임시 리스트에 저장한다
    for i in range(len(EX1_represent_Ac_std.columns)):
        
        #한 사람이 어떤 과목을 봤을지 모르므로 다 확인해 보는게 맞다
        if EX1_represent_Ac_std.columns[i] in list(Sumind['SUB_CD']):
                    EX1_represent_Ac_std.loc[EX1_represent_Ac_std['MEM_NO']==mem_no,EX1_represent_Ac_std.columns[i]] = Sumind[EX1_represent_Ac_std.columns[i]==Sumind['SUB_CD']]['AC_STD'].iloc[0] #컬럼과 뽑아낸 과목코드가 같은 것의 값을

In [ ]:
for mem_no in EX2_represent_Ac_std['MEM_NO']:
    Sumind = sampledata_ex2[sampledata_ex2['MEM_NO']==mem_no]['AC_STD'].groupby(sampledata_ex2['SUB_CD']).mean().add_suffix('').reset_index()
    #대표값을 만들어 임시 리스트에 저장한다
    for i in range(len(EX2_represent_Ac_std.columns)):
        
        #한 사람이 어떤 과목을 봤을지 모르므로 다 확인해 보는게 맞다
        if EX2_represent_Ac_std.columns[i] in list(Sumind['SUB_CD']):
                    EX2_represent_Ac_std.loc[EX2_represent_Ac_std['MEM_NO']==mem_no,EX2_represent_Ac_std.columns[i]] = Sumind[EX2_represent_Ac_std.columns[i]==Sumind['SUB_CD']]['AC_STD'].iloc[0] #컬럼과 뽑아낸 과목코드가 같은 것의 값을

# 위가 실행되면 반드시 아래 저장도 실행되어야함

In [14]:
#EX1_represent.to_csv(r'C:\Users\74857\데이콘야구\동국대\EX1_represent_Ac.txt',header=True,index=False,sep=',')
#EX2_represent.to_csv(r'C:\Users\74857\데이콘야구\동국대\EX2_represent_Ac.txt',header=True,index=False,sep=',')

설정 가능한 것: [1000,2100,2200,3000,4102,4111,4116,4201,4203,4207,4213]

In [20]:
EX1_inter = pd.read_csv(r'C:\Users\74857\데이콘야구\동국대\EX1_represent_AC.txt', engine='python')
EX2_inter = pd.read_csv(r'C:\Users\74857\데이콘야구\동국대\EX2_represent_AC.txt', engine='python')

In [47]:
EX1_inter.head()

,MEM_NO,1000,2100,2200,3000,4102,4103,4107,4110,4111,...,5001,5002,5003,5004,5005,5006,5007,5008,5009,7000
0,14402,27.0,31.5,19.0,32.333333,NaN,NaN,NaN,NaN,22.0,...,39.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.333333
1,69420,8.0,NaN,78.0,11.000000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000
2,77385,71.0,NaN,64.0,55.000000,79.0,60.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,58.000000
3,83920,NaN,NaN,100.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,101936,89.5,NaN,84.0,76.500000,82.0,49.5,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,98.000000


In [122]:
truedata = []
canlist = [1000,2100,2200,3000,4102,4111,4116,4201,4203,4207,4213]
for i in canlist:
    truedata.append(pd.read_csv(r'C:\Users\74857\데이콘야구\동국대\데이터\데이터\x'+str(i)+'.csv', engine='python'))

In [65]:
EX1_wecan = EX1_inter.loc[:,['MEM_NO']+[str(i) for i in canlist]].copy()
EX2_wecan = EX2_inter.loc[:,['MEM_NO']+[str(i) for i in canlist]].copy()

In [66]:
EX2_wecan.describe()

,MEM_NO,1000,2100,2200,3000,4102,4111,4116,4201,4203,4207,4213
count,1.652050e+05,164258.000000,79886.000000,89328.000000,163193.000000,25456.000000,43922.000000,46038.000000,26071.00000,37428.000000,64729.000000,57407.000000
mean,1.772715e+12,44.414024,43.775946,44.533200,78.691438,45.410778,44.465361,44.967133,44.46517,43.668801,44.043547,43.783305
std,2.160414e+11,27.298169,27.302817,27.252565,17.608448,27.361962,27.023577,27.125114,27.31708,26.892559,26.851623,26.730504
min,2.251019e+06,0.000000,0.000000,0.000000,30.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
25%,1.756814e+12,21.000000,20.000000,21.000000,70.000000,22.000000,21.500000,22.000000,21.00000,20.000000,21.000000,21.000000
50%,1.776841e+12,43.000000,42.000000,43.000000,81.250000,44.000000,43.000000,44.000000,42.00000,42.000000,42.000000,42.000000
75%,1.846082e+12,66.333333,65.833333,66.600000,92.500000,68.000000,67.000000,67.000000,67.00000,65.000000,66.000000,65.000000
max,1.886815e+12,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.00000,100.000000,100.000000,100.000000


In [67]:
EX2_wecan.loc[:'1000'] = 0.49081*EX2_wecan.loc[:'1000'] + 47.47992
EX2_wecan.loc[:'2100'] = 0.49159*EX2_wecan.loc[:'2100'] + 44.46125
EX2_wecan.loc[:'2200'] = 0.41496*EX2_wecan.loc[:'2200'] + 55.41040
EX2_wecan.loc[:'3000'] = 1.17835*EX2_wecan.loc[:'3000'] - 32.37653
EX2_wecan.loc[:'4102'] = 0.34751*EX2_wecan.loc[:'4102'] + 59.69838
EX2_wecan.loc[:'4111'] = 0.41772*EX2_wecan.loc[:'4111'] + 52.59915
EX2_wecan.loc[:'4116'] = 0.23965*EX2_wecan.loc[:'4116'] + 66.20862
EX2_wecan.loc[:'4201'] = 0.39509*EX2_wecan.loc[:'4201'] + 54.20256
EX2_wecan.loc[:'4203'] = 0.3615*EX2_wecan.loc[:'4203'] + 58.6581
EX2_wecan.loc[:'4207'] = 0.54883*EX2_wecan.loc[:'4207'] + 43.14528
EX2_wecan.loc[:'4213'] = 0.38774*EX2_wecan.loc[:'4213'] + 56.06712

In [68]:
EX2_wecan.describe()

,MEM_NO,1000,2100,2200,3000,4102,4111,4116,4201,4203,4207,4213
count,1.652050e+05,164258.000000,79886.000000,89328.000000,163193.000000,25456.000000,43922.000000,46038.000000,26071.000000,37428.000000,64729.000000,57407.000000
mean,1.327164e+12,55.970329,56.876708,55.348691,82.465376,57.988345,56.364096,56.720452,57.455910,57.234301,55.624643,55.610089
std,8.038867e+11,31.780736,32.116437,31.544617,15.891335,31.637594,31.640425,31.576604,31.997255,32.054743,31.645952,31.590586
min,2.251019e+06,0.000000,0.000000,0.000000,30.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.212885e+09,27.000000,27.000000,27.000000,75.000000,29.500000,28.000000,28.000000,28.000000,27.000000,26.000000,26.000000
50%,1.776158e+12,58.000000,59.875000,56.500000,90.000000,61.125000,58.000000,59.000000,60.750000,60.000000,57.000000,56.000000
75%,1.846082e+12,92.033112,92.047077,92.025118,92.108735,92.056374,92.039456,92.041571,92.060603,92.055020,92.030101,92.037342
max,1.886815e+12,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000


In [80]:
EX2_wecan = EX2_wecan[~EX2_wecan['MEM_NO'].isin(intersut)]

In [81]:
EX_fusion = pd.concat([EX1_wecan,EX2_wecan])

In [82]:
EX_fusion.head()

,MEM_NO,1000,2100,2200,3000,4102,4111,4116,4201,4203,4207,4213
0,14402.0,27.0,31.5,19.0,32.333333,NaN,22.0,18.0,NaN,NaN,23.0,23.0
1,69420.0,8.0,NaN,78.0,11.000000,NaN,NaN,99.0,NaN,NaN,NaN,NaN
2,77385.0,71.0,NaN,64.0,55.000000,79.0,NaN,NaN,NaN,NaN,NaN,NaN
3,83920.0,NaN,NaN,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,101936.0,89.5,NaN,84.0,76.500000,82.0,NaN,NaN,NaN,NaN,NaN,NaN


In [83]:
len(EX_fusion['MEM_NO'])

220993

In [85]:
suneglist = [222,223, 238,239]
BIG3_elidata[(BIG3_elidata['EXAMNO'].isin(suneglist))]['MEM_NO'].unique()

array([ 358458,  413694,  700438, ..., 6285134, 6291967, 6292407],
      dtype=int64)

In [87]:
len(BIG3_elidata[(BIG3_elidata['EXAMNO'].isin(suneglist))]['MEM_NO'].unique())

27908

In [89]:
len(EX_fusion[EX_fusion['MEM_NO'].isin(BIG3_elidata[(BIG3_elidata['EXAMNO'].isin(suneglist))]['MEM_NO'].unique())])

27908

# 여기 있는 사람 모두 수능은 보았음.

In [120]:
def Concept_Rating(data,col_name='PURE_RATING' ,SCORE='SCORE'):
    SOCRE_percentile_list = [np.nanpercentile(data[SCORE],96),
                         np.nanpercentile(data[SCORE],89),
                         np.nanpercentile(data[SCORE],77),
                         np.nanpercentile(data[SCORE],60),
                         np.nanpercentile(data[SCORE],40),
                         np.nanpercentile(data[SCORE],23),
                         np.nanpercentile(data[SCORE],11),
                         np.nanpercentile(data[SCORE],4),
                        ]
    data[col_name] = None
    data.loc[data[SCORE]>=SOCRE_percentile_list[0],col_name]=1
    data.loc[(data[SCORE]<SOCRE_percentile_list[0]) & (data[SCORE]>=SOCRE_percentile_list[1]),col_name]=2
    data.loc[(data[SCORE]<SOCRE_percentile_list[1]) & (data[SCORE]>=SOCRE_percentile_list[2]),col_name]=3
    data.loc[(data[SCORE]<SOCRE_percentile_list[2]) & (data[SCORE]>=SOCRE_percentile_list[3]),col_name]=4
    data.loc[(data[SCORE]<SOCRE_percentile_list[3]) & (data[SCORE]>=SOCRE_percentile_list[4]),col_name]=5
    data.loc[(data[SCORE]<SOCRE_percentile_list[4]) & (data[SCORE]>=SOCRE_percentile_list[5]),col_name]=6
    data.loc[(data[SCORE]<SOCRE_percentile_list[5]) & (data[SCORE]>=SOCRE_percentile_list[6]),col_name]=7
    data.loc[(data[SCORE]<SOCRE_percentile_list[6]) & (data[SCORE]>=SOCRE_percentile_list[7]),col_name]=8
    data.loc[data[SCORE]<SOCRE_percentile_list[7],col_name]=9
    
    return data

In [123]:
for i in canlist:
    EX_fusion = Concept_Rating(EX_fusion,col_name = "RATING of"+str(i),SCORE = str(i))

In [111]:
np.nanpercentile(EX_fusion['1000'],96)

95.0

In [162]:
EX_fusion[EX_fusion['RATING of1000']==3.0][['MEM_NO','RATING of1000']]

,MEM_NO,RATING of1000
56380,434891401,3
56384,434891401,3
56387,434891401,3
56388,434891402,3
56390,434891402,3
56399,434891402,3
56413,434891403,3
56421,434900042,3
56425,434900042,3
56426,434900042,3


In [129]:
EX_fusion.iloc[:,[0,1,12]].head()

,MEM_NO,1000,RATING of1000
0,14402,27.0,7
1,69420,8.0,8
2,77385,71.0,5
3,83920,NaN,None
4,101936,89.5,4


In [145]:
#EX_fusion.to_csv(r'C:\Users\74857\데이콘야구\동국대\EX_fusion.csv',header=True,index=False,sep=',')
#EX_fusion = pd.read_csv(r'C:\Users\74857\데이콘야구\동국대\EX_fusion.csv',engine='python')
#EX_fusion2.to_csv(r'C:\Users\74857\데이콘야구\동국대\EX_fusion2.csv',header=True,index=False,sep=',')

# 이제 수능 점수에 대해
- 평균 백분위가 주어졌을 때, P(B) : 평균 백분위의 분포
- 수능 등급의 분포 P(S | B) 를 구해보자
P(S and B) / P(B)

In [14]:
#final_score = pd.read_csv(r'C:\Users\74857\데이콘야구\동국대\score.csv',engine='python')
#final_score2.to_csv(r'C:\Users\74857\데이콘야구\동국대\score2.csv',header=True,index=False,sep=',')
#final_score = pd.read_csv(r'C:\Users\74857\데이콘야구\동국대\score2.csv',engine='python')
final_score2 = pd.read_csv(r'C:\Users\74857\데이콘야구\동국대\EX_fusion2.csv',engine='python')

In [150]:
final_score2.columns # 그냥과목코드 : 백분위평균 대표값, RAITNG : 비율을 고려치 않고 전체에 적용한 RAITNG, 18RAITNG : 수능 등급

Index(['MEM_NO', 'EXAMTYPE', '1000', '2100', '2200', '3000', '4102', '4111',
       '4116', '4201', '4203', '4207', '4213', 'RATING of1000',
       'RATING of2100', 'RATING of2200', 'RATING of3000', 'RATING of4102',
       'RATING of4111', 'RATING of4116', 'RATING of4201', 'RATING of4203',
       'RATING of4207', 'RATING of4213', '17RATING_1000', '17RATING_2100',
       '17RATING_2200', '17RATING_3000', '17RATING_4102', '17RATING_4111',
       '17RATING_4116', '17RATING_4201', '17RATING_4203', '17RATING_4207',
       '17RATING_4213', '18RATING_1000', '18RATING_2100', '18RATING_2200',
       '18RATING_3000', '18RATING_4102', '18RATING_4111', '18RATING_4116',
       '18RATING_4201', '18RATING_4203', '18RATING_4207', '18RATING_4213'],
      dtype='object')

In [9]:
MEMEX = BIG3_elidata['SCORE'].groupby([BIG3_elidata['EXAMTYPE'],BIG3_elidata['MEM_NO']]).mean().add_suffix('').reset_index()[['EXAMTYPE','MEM_NO']]

In [39]:
str(int(final_score['MEM_NO'][0]))

'10951'

In [25]:
final_score['MEM_NO'] = [str(int(num))  for num in final_score['MEM_NO']]
EX_fusion['MEM_NO'] = [str(int(num))  for num in EX_fusion['MEM_NO']]

In [141]:
final_score2 = pd.merge(final_score, MEMEX, how='left' ,on = 'MEM_NO')[[
    'MEM_NO', 'EXAMTYPE', 'X1000', 'X2100', 'X2200', 'X3000', 'X4102',
       'X4111', 'X4116', 'X4201', 'X4203', 'X4207', 'X4213', 'RATING.of1000',
       'RATING.of2100', 'RATING.of2200', 'RATING.of3000', 'RATING.of4102',
       'RATING.of4111', 'RATING.of4116', 'RATING.of4201', 'RATING.of4203',
       'RATING.of4207', 'RATING.of4213', '17RATING_1000', '17RATING_2100',
       '17RATING_2200', '17RATING_3000', '17RATING_4102', '17RATING_4111',
       '17RATING_4116', '17RATING_4201', '17RATING_4203', '17RATING_4207',
       '17RATING_4213', '18RATING_1000', '18RATING_2100', '18RATING_2200',
       '18RATING_3000', '18RATING_4102', '18RATING_4111', '18RATING_4116',
       '18RATING_4201', '18RATING_4203', '18RATING_4207', '18RATING_4213']]

EX_fusion2 = pd.merge(EX_fusion, MEMEX, how='left' ,on = 'MEM_NO')[[
    'MEM_NO','EXAMTYPE' , '1000', '2100', '2200', '3000', '4102', '4111', '4116',
       '4201', '4203', '4207', '4213', 'RATING of1000', 'RATING of2100',
       'RATING of2200', 'RATING of3000', 'RATING of4102', 'RATING of4111',
       'RATING of4116', 'RATING of4201', 'RATING of4203', 'RATING of4207',
       'RATING of4213']]

EX_fusion2 = pd.merge(EX_fusion2, final_score2, how='left' ,on = 'MEM_NO')[[
    'MEM_NO', '1000', '2100', '2200', '3000', '4102', '4111',
       '4116', '4201', '4203', '4207', '4213', 'RATING of1000',
       'RATING of2100', 'RATING of2200', 'RATING of3000', 'RATING of4102',
       'RATING of4111', 'RATING of4116', 'RATING of4201', 'RATING of4203',
       'RATING of4207', 'RATING of4213',
       '17RATING_1000', '17RATING_2100',
       '17RATING_2200', '17RATING_3000', '17RATING_4102', '17RATING_4111',
       '17RATING_4116', '17RATING_4201', '17RATING_4203', '17RATING_4207',
       '17RATING_4213', '18RATING_1000', '18RATING_2100', '18RATING_2200',
       '18RATING_3000', '18RATING_4102', '18RATING_4111', '18RATING_4116',
       '18RATING_4201', '18RATING_4203', '18RATING_4207', '18RATING_4213'
    
]]

In [144]:
EX_fusion2 = pd.merge(EX_fusion2, MEMEX, how='left' ,on = 'MEM_NO')[[
    'MEM_NO','EXAMTYPE', '1000', '2100', '2200', '3000', '4102', '4111',
       '4116', '4201', '4203', '4207', '4213', 'RATING of1000',
       'RATING of2100', 'RATING of2200', 'RATING of3000', 'RATING of4102',
       'RATING of4111', 'RATING of4116', 'RATING of4201', 'RATING of4203',
       'RATING of4207', 'RATING of4213',
       '17RATING_1000', '17RATING_2100',
       '17RATING_2200', '17RATING_3000', '17RATING_4102', '17RATING_4111',
       '17RATING_4116', '17RATING_4201', '17RATING_4203', '17RATING_4207',
       '17RATING_4213', '18RATING_1000', '18RATING_2100', '18RATING_2200',
       '18RATING_3000', '18RATING_4102', '18RATING_4111', '18RATING_4116',
       '18RATING_4201', '18RATING_4203', '18RATING_4207', '18RATING_4213'
    
]]

In [174]:
EX_fusion2 = EX_fusion2.drop_duplicates('MEM_NO',keep='last')

In [175]:
final_score2 = EX_fusion2

In [176]:
Data_2017 = final_score2[['MEM_NO', 'EXAMTYPE', '1000', '2100', '2200', '3000', '4102', '4111',
       '4116', '4201', '4203', '4207', '4213', 'RATING of1000',
       'RATING of2100', 'RATING of2200', 'RATING of3000', 'RATING of4102',
       'RATING of4111', 'RATING of4116', 'RATING of4201', 'RATING of4203',
       'RATING of4207', 'RATING of4213', '17RATING_1000', '17RATING_2100',
       '17RATING_2200', '17RATING_3000', '17RATING_4102', '17RATING_4111',
       '17RATING_4116', '17RATING_4201', '17RATING_4203', '17RATING_4207',
       '17RATING_4213']].copy()
Data_2018 = final_score2[['MEM_NO', 'EXAMTYPE', '1000', '2100', '2200', '3000', '4102', '4111',
       '4116', '4201', '4203', '4207', '4213', 'RATING of1000',
       'RATING of2100', 'RATING of2200', 'RATING of3000', 'RATING of4102',
       'RATING of4111', 'RATING of4116', 'RATING of4201', 'RATING of4203',
       'RATING of4207', 'RATING of4213','18RATING_1000', '18RATING_2100', '18RATING_2200',
       '18RATING_3000', '18RATING_4102', '18RATING_4111', '18RATING_4116',
       '18RATING_4201', '18RATING_4203', '18RATING_4207', '18RATING_4213']].copy()


In [49]:
len(final_score[(~final_score['RATING.of1000'].isnull()) & (~final_score['18RATING_1000'].isnull())])

9049

In [178]:
Data_2018.loc[Data_2018['EXAMTYPE'].isnull(),'EXAMTYPE']=2

In [154]:
def MakeTSMatrix(dataDF ,num ,colname = 'RATING of' , Compare = '18RATING_'):
    tempdata = dataDF[(~dataDF[colname+num].isnull()) & (~dataDF[Compare+num].isnull()) ]
    Rating_Change_Matrix = np.zeros((9,9))
    for i in range(1,10):
        if len(tempdata[ (tempdata[colname+num]==i)])==0:
                print(colname+'중 '+str(i)+'등급은 인원이 없습니다')
        for j in range(1,10):
            if len(tempdata[ (tempdata[colname+num]==i)])==0:
                pass
            else:
                Rating_Change_Matrix[i-1][j-1]=round(len(tempdata[(tempdata[colname+num]==i) & (tempdata[Compare+num]==j)]
                                              )/len(tempdata[ (tempdata[colname+num]==i)]),2)
    
    return Rating_Change_Matrix

In [155]:
subject = ['1000','2100','2200','3000','4102','4111','4116','4201','4203','4207','4213']

In [190]:
test = MakeTSMatrix(Data_2018 , '1000', colname = 'RATING of', Compare = '18RATING_')

RATING of중 3등급은 인원이 없습니다


In [191]:
test

array([[0.96, 0.04, 0.01, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.43, 0.48, 0.09, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.02, 0.23, 0.55, 0.2 , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.01, 0.03, 0.56, 0.39, 0.01, 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.02, 0.45, 0.5 , 0.01, 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.01, 0.37, 0.58, 0.03, 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.01, 0.49, 0.46, 0.04],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.5 , 0.5 ]])

In [18]:
test2 = []
for i in subject:
    test2.append(MakeTSMatrix(Data_2018 , i , colname = 'RATING.of', Compare = '18RATING_'))

RATING.of중 1등급은 인원이 없습니다
RATING.of중 2등급은 인원이 없습니다
RATING.of중 3등급은 인원이 없습니다
RATING.of중 1등급은 인원이 없습니다
RATING.of중 2등급은 인원이 없습니다
RATING.of중 3등급은 인원이 없습니다
RATING.of중 1등급은 인원이 없습니다
RATING.of중 2등급은 인원이 없습니다
RATING.of중 3등급은 인원이 없습니다
RATING.of중 1등급은 인원이 없습니다
RATING.of중 2등급은 인원이 없습니다
RATING.of중 1등급은 인원이 없습니다
RATING.of중 2등급은 인원이 없습니다
RATING.of중 3등급은 인원이 없습니다
RATING.of중 1등급은 인원이 없습니다
RATING.of중 2등급은 인원이 없습니다
RATING.of중 3등급은 인원이 없습니다
RATING.of중 1등급은 인원이 없습니다
RATING.of중 2등급은 인원이 없습니다
RATING.of중 3등급은 인원이 없습니다
RATING.of중 1등급은 인원이 없습니다
RATING.of중 2등급은 인원이 없습니다
RATING.of중 3등급은 인원이 없습니다
RATING.of중 1등급은 인원이 없습니다
RATING.of중 2등급은 인원이 없습니다
RATING.of중 3등급은 인원이 없습니다
RATING.of중 1등급은 인원이 없습니다
RATING.of중 2등급은 인원이 없습니다
RATING.of중 3등급은 인원이 없습니다
RATING.of중 1등급은 인원이 없습니다
RATING.of중 2등급은 인원이 없습니다
RATING.of중 3등급은 인원이 없습니다


In [28]:
for i in range(len(subject)):
    print(test2[i])
    print("2018년의 과목코드 "+str(subject[i])+ "의 확률행렬\n")

[[0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.03 0.21 0.59 0.16 0.01 0.   0.   0.   0.  ]
 [0.01 0.01 0.06 0.54 0.35 0.02 0.   0.   0.  ]
 [0.   0.02 0.01 0.04 0.44 0.43 0.03 0.01 0.02]
 [0.   0.   0.   0.   0.04 0.37 0.54 0.04 0.01]
 [0.   0.   0.   0.   0.   0.01 0.2  0.73 0.06]
 [0.   0.   0.   0.   0.   0.   0.   0.   1.  ]]
2018년의 과목코드 1000의 확률행렬

[[0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.05 0.29 0.51 0.13 0.02 0.   0.   0.   0.  ]
 [0.01 0.02 0.05 0.52 0.36 0.04 0.   0.   0.  ]
 [0.   0.01 0.   0.02 0.44 0.47 0.03 0.   0.01]
 [0.   0.   0.   0.   0.02 0.37 0.58 0.03 0.01]
 [0.   0.   0.   0.   0.   0.01 0.16 0.74 0.09]
 [0.   0.   0.   0.   0.   0.   0.   0.01 0.99]]
2018년의 과목코드 2100의 확률행렬

[[0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0

In [29]:
test3 = []
for i in subject:
    test3.append(MakeTSMatrix(Data_2017 , i , colname = 'RATING.of', Compare = '17RATING_'))

RATING.of중 1등급은 인원이 없습니다
RATING.of중 2등급은 인원이 없습니다
RATING.of중 3등급은 인원이 없습니다
RATING.of중 1등급은 인원이 없습니다
RATING.of중 2등급은 인원이 없습니다
RATING.of중 3등급은 인원이 없습니다
RATING.of중 1등급은 인원이 없습니다
RATING.of중 2등급은 인원이 없습니다
RATING.of중 3등급은 인원이 없습니다
RATING.of중 1등급은 인원이 없습니다
RATING.of중 2등급은 인원이 없습니다
RATING.of중 1등급은 인원이 없습니다
RATING.of중 2등급은 인원이 없습니다
RATING.of중 3등급은 인원이 없습니다
RATING.of중 1등급은 인원이 없습니다
RATING.of중 2등급은 인원이 없습니다
RATING.of중 3등급은 인원이 없습니다
RATING.of중 1등급은 인원이 없습니다
RATING.of중 2등급은 인원이 없습니다
RATING.of중 3등급은 인원이 없습니다
RATING.of중 1등급은 인원이 없습니다
RATING.of중 2등급은 인원이 없습니다
RATING.of중 3등급은 인원이 없습니다
RATING.of중 1등급은 인원이 없습니다
RATING.of중 2등급은 인원이 없습니다
RATING.of중 3등급은 인원이 없습니다
RATING.of중 1등급은 인원이 없습니다
RATING.of중 2등급은 인원이 없습니다
RATING.of중 3등급은 인원이 없습니다
RATING.of중 1등급은 인원이 없습니다
RATING.of중 2등급은 인원이 없습니다
RATING.of중 3등급은 인원이 없습니다


In [31]:
for i in range(len(subject)):
    print(test3[i])
    print("2017년의 과목코드 "+str(subject[i])+ "의 확률행렬\n")

[[0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.04 0.24 0.6  0.1  0.01 0.   0.   0.   0.  ]
 [0.01 0.01 0.04 0.54 0.37 0.02 0.   0.   0.01]
 [0.   0.   0.01 0.02 0.43 0.48 0.04 0.   0.02]
 [0.   0.   0.   0.01 0.02 0.38 0.54 0.04 0.02]
 [0.   0.   0.   0.   0.01 0.   0.35 0.53 0.11]
 [0.   0.   0.   0.   0.   0.   0.   0.   1.  ]]
2017년의 과목코드 1000의 확률행렬

[[0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.03 0.44 0.44 0.07 0.03 0.   0.   0.   0.  ]
 [0.02 0.03 0.07 0.37 0.42 0.06 0.   0.   0.02]
 [0.   0.   0.01 0.01 0.38 0.55 0.04 0.   0.01]
 [0.   0.   0.   0.   0.01 0.38 0.56 0.02 0.02]
 [0.   0.   0.   0.   0.   0.   0.11 0.77 0.12]
 [0.   0.   0.   0.   0.   0.   0.   0.   1.  ]]
2017년의 과목코드 2100의 확률행렬

[[0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0

# 이제 결국 비율을 조절해서 해봐야한다

In [300]:
def MakeRank_by_rating(data,SUB_CODE,flag=False):
    #flag는 False면 Extype=1 기준, True면 Extype 2 기준
    dataDF= data[['MEM_NO','EXAMTYPE',SUB_CODE,'RATING of'+SUB_CODE]]
    #시험 코드 대로 데이터프레임을 하나 만든다.
    if flag==False:
        dataDF_0 = dataDF[dataDF['EXAMTYPE']=='1']
        dataDF_1 = dataDF[dataDF['EXAMTYPE']=='2']
    if flag==True:
        dataDF_0 = dataDF[dataDF['EXAMTYPE']=='2']
        dataDF_1 = dataDF[dataDF['EXAMTYPE']=='1']
    
    #학원생의 데이터집합을 만든다
    dataDF_0 = Concept_Rating(dataDF_0,SCORE = SUB_CODE)
    #학원생끼리 다시 점수를 기반으로 레이팅한다. ->함수화
    
    N_rate = 0.7 #최대치
    already_sample = rd.sample(range(len(dataDF_1)),int(np.round(len(dataDF_0)*(1-N_rate)/(N_rate))))
    
    N_rate = 0.95 #비학원생이 95%면 좋겠다는 뜻
    rate_len1 = int(np.round(len(dataDF_0)*(1-N_rate)/(N_rate)))
    Compare = 'N'+str(int(N_rate*100))+':Y'+str(100-int(N_rate*100))
    dataDF_X= pd.concat([dataDF_0,dataDF_1.iloc[already_sample[:rate_len1],:]],sort=False)
    dataDF_X = Concept_Rating(dataDF_X,Compare,SCORE = SUB_CODE)
    
    
    N_rate = 0.9 #비학원생이 90%면 좋겠다는 뜻
    rate_len2 = int(np.round(len(dataDF_0)*(1-N_rate)/(N_rate)))-rate_len1
    Compare = 'N'+str(int(N_rate*100))+':Y'+str(100-int(N_rate*100))
    dataDF_X= pd.concat([dataDF_X,dataDF_1.iloc[already_sample[rate_len1:rate_len2],:]],sort=False)#여기서 이미 추가된 애들을 제외한다.
    dataDF_X = Concept_Rating(dataDF_X,Compare,SCORE = SUB_CODE)
    
    N_rate = 0.85 
    rate_len3 = int(np.round(len(dataDF_0)*(1-N_rate)/(N_rate)))-rate_len2
    Compare = 'N'+str(int(N_rate*100))+':Y'+str(100-int(N_rate*100))
    dataDF_X= pd.concat([dataDF_X,dataDF_1.iloc[already_sample[rate_len2:rate_len3],:]],sort=False)#여기서 이미 추가된 애들을 제외한다.
    dataDF_X = Concept_Rating(dataDF_X,Compare,SCORE = SUB_CODE)
    
    N_rate = 0.8 
    rate_len4 = int(np.round(len(dataDF_0)*(1-N_rate)/(N_rate)))-rate_len3
    Compare = 'N'+str(int(N_rate*100))+':Y'+str(100-int(N_rate*100))
    dataDF_X= pd.concat([dataDF_X,dataDF_1.iloc[already_sample[rate_len3:rate_len4],:]],sort=False)#여기서 이미 추가된 애들을 제외한다.
    dataDF_X = Concept_Rating(dataDF_X,Compare,SCORE = SUB_CODE)
    
    N_rate = 0.75
    rate_len5 = int(np.round(len(dataDF_0)*(1-N_rate)/(N_rate)))-rate_len4
    Compare = 'N'+str(int(N_rate*100))+':Y'+str(100-int(N_rate*100))
    dataDF_X= pd.concat([dataDF_X,dataDF_1.iloc[already_sample[rate_len4:rate_len5],:]],sort=False)#여기서 이미 추가된 애들을 제외한다.
    dataDF_X = Concept_Rating(dataDF_X,Compare,SCORE = SUB_CODE)
    
    N_rate = 0.7 
    rate_len6 = int(np.round(len(dataDF_0)*(1-N_rate)/(N_rate)))-rate_len5
    Compare = 'N'+str(int(N_rate*100))+':Y'+str(100-int(N_rate*100))
    dataDF_X= pd.concat([dataDF_X,dataDF_1.iloc[already_sample[rate_len5:rate_len6],:]],sort=False)#여기서 이미 추가된 애들을 제외한다.
    dataDF_X = Concept_Rating(dataDF_X,Compare,SCORE = SUB_CODE)
    
    
    
    return dataDF_X


In [301]:
EX_2100_flag1 = MakeRank_by_rating(EX_fusion2,'2100',flag=False)
EX_2100_flag2 = MakeRank_by_rating(EX_fusion2,'2100',flag=True)

C:\Users\74857\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


In [294]:
EX_1000_flag1.columns

Index(['MEM_NO', 'EXAMTYPE', '1000', 'RATING of1000', 'PURE_RATING', 'N95:Y5',
       'N90:Y10', 'N85:Y15', 'N80:Y20', 'N75:Y25', 'N70:Y30', 'N65:Y35',
       'N60:Y40'],
      dtype='object')

In [304]:
#EX_2100_2_flag1 = pd.merge(EX_2100_flag1,Data_2018[['MEM_NO','18RATING_2100']],on = 'MEM_NO')
#EX_2100_2_flag1 = EX_2100_2_flag1[(~EX_2100_2_flag1['2100'].isnull()) & (~EX_2100_2_flag1['18RATING_2100'].isnull())]

EX_2100_2_flag2 = pd.merge(EX_2100_flag2,Data_2018[['MEM_NO','18RATING_2100']],on = 'MEM_NO')
EX_2100_2_flag2 = EX_2100_2_flag2[(~EX_2100_2_flag2['2100'].isnull()) & (~EX_2100_2_flag2['18RATING_2100'].isnull())]

In [295]:
import sklearn.metrics as metrics #1 기준
for i in ['PURE_RATING', 'N95:Y5',
       'N90:Y10', 'N85:Y15', 'N80:Y20', 'N75:Y25', 'N70:Y30', 'N65:Y35',
       'N60:Y40']:
    print(i+"의 경우:")
    print('accuracy\n', metrics.accuracy_score(EX_1000_2_flag1[i].astype('str'),EX_1000_2_flag1['18RATING_1000'].astype('int').astype('str') ))

PURE_RATING의 경우:
accuracy
 0.13537296633582319
N95:Y5의 경우:
accuracy
 0.1466284661823391
N90:Y10의 경우:
accuracy
 0.1619768750639517
N85:Y15의 경우:
accuracy
 0.17916709301135783
N80:Y20의 경우:
accuracy
 0.19400388826358334
N75:Y25의 경우:
accuracy
 0.26593676455540777
N70:Y30의 경우:
accuracy
 0.27402025989972373
N65:Y35의 경우:
accuracy
 0.33193492274634195
N60:Y40의 경우:
accuracy
 0.3772638903100379


In [306]:
for i in ['RATING of1000', 'PURE_RATING', 'N95:Y5', # 2기준
       'N90:Y10', 'N85:Y15', 'N80:Y20', 'N75:Y25', 'N70:Y30']:
    print(i+"의 경우:")
    print('accuracy\n', metrics.accuracy_score(EX_1000_2_flag2[i].astype('str'),EX_1000_2_flag2['18RATING_1000'].astype('int').astype('str')) )

RATING of1000의 경우:
accuracy
 0.4064945226917058
PURE_RATING의 경우:
accuracy
 0.02112676056338028
N95:Y5의 경우:
accuracy
 0.14456181533646323
N90:Y10의 경우:
accuracy
 0.15923317683881064
N85:Y15의 경우:
accuracy
 0.32316118935837246
N80:Y20의 경우:
accuracy
 0.3765649452269171
N75:Y25의 경우:
accuracy
 0.6056338028169014
N70:Y30의 경우:
accuracy
 0.6784037558685446


In [307]:
for i in ['RATING of2100', 'PURE_RATING', 'N95:Y5', # 2기준
       'N90:Y10', 'N85:Y15', 'N80:Y20', 'N75:Y25', 'N70:Y30']:
    print(i+"의 경우:")
    print('accuracy\n', metrics.accuracy_score(EX_2100_2_flag2[i].astype('str'),EX_2100_2_flag2['18RATING_2100'].astype('int').astype('str')) )

RATING of2100의 경우:
accuracy
 0.4197475202885482
PURE_RATING의 경우:
accuracy
 0.022091974752028856
N95:Y5의 경우:
accuracy
 0.14833183047790802
N90:Y10의 경우:
accuracy
 0.1654643823264202
N85:Y15의 경우:
accuracy
 0.3097385031559964
N80:Y20의 경우:
accuracy
 0.3394950405770965
N75:Y25의 경우:
accuracy
 0.5302073940486925
N70:Y30의 경우:
accuracy
 0.6055004508566276


# EXtype=2 학생 70 %: EXtype=1 학생 30% 일때 오히려 가장 잘 맞는다

In [210]:
def MakeTSMatrix2(dataDF ,colname , Compare ):
    tempdata = dataDF[(~dataDF[colname].isnull()) & (~dataDF[Compare].isnull()) ]
    Rating_Change_Matrix = np.zeros((9,9))
    for i in range(1,10):
        if len(tempdata[ (tempdata[colname]==i)])==0:
                print(colname+'중 '+str(i)+'등급은 인원이 없습니다')
        for j in range(1,10):
            if len(tempdata[ (tempdata[colname]==i)])==0:
                pass
            else:
                Rating_Change_Matrix[i-1][j-1]=round(len(tempdata[(tempdata[colname]==i) & (tempdata[Compare]==j)]
                                              )/len(tempdata[ (tempdata[colname]==i)]),2)
    
    return Rating_Change_Matrix

In [278]:
test_3 = MakeTSMatrix2(EX_1000_2_flag2 , colname = 'N75:Y25', Compare = '18RATING_1000')
print(test_3)

[[0.82 0.18 0.   0.   0.   0.   0.   0.   0.  ]
 [0.1  0.55 0.34 0.01 0.   0.   0.   0.   0.  ]
 [0.01 0.05 0.61 0.31 0.01 0.   0.   0.   0.  ]
 [0.01 0.01 0.05 0.65 0.27 0.01 0.   0.   0.  ]
 [0.   0.01 0.02 0.05 0.7  0.2  0.01 0.01 0.01]
 [0.   0.   0.   0.02 0.04 0.75 0.17 0.01 0.01]
 [0.   0.   0.   0.   0.   0.05 0.79 0.15 0.01]
 [0.   0.   0.   0.   0.   0.02 0.   0.84 0.15]
 [0.   0.   0.   0.   0.   0.   0.   0.   1.  ]]


# EXTYPE=2 로만 구성했을 시, 실제 수능에서의 등급은 오히려 밀린다!

In [248]:
test_3 = MakeTSMatrix2(EX_1000_2_flag2 , colname = 'PURE_RATING', Compare = '18RATING_1000')
print(test_3)

[[0.71 0.24 0.05 0.   0.   0.   0.   0.   0.  ]
 [0.08 0.3  0.59 0.03 0.   0.   0.   0.   0.  ]
 [0.02 0.07 0.34 0.53 0.03 0.   0.   0.   0.  ]
 [0.   0.03 0.13 0.42 0.35 0.03 0.   0.   0.03]
 [0.   0.   0.11 0.11 0.67 0.   0.   0.   0.11]
 [0.   0.   0.   0.   0.   0.5  0.25 0.25 0.  ]
 [0.   0.   0.   0.   0.   1.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.   1.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.   1.  ]]


In [250]:
test_3 = MakeTSMatrix2(EX_1000_2_flag2 , colname = 'N70:Y30', Compare = '18RATING_1000')
print(test_3)

[[0.84 0.16 0.   0.   0.   0.   0.   0.   0.  ]
 [0.1  0.55 0.33 0.01 0.   0.   0.   0.   0.  ]
 [0.01 0.05 0.62 0.3  0.01 0.   0.   0.   0.  ]
 [0.01 0.01 0.05 0.65 0.27 0.01 0.   0.   0.  ]
 [0.   0.01 0.02 0.05 0.7  0.2  0.01 0.01 0.01]
 [0.   0.   0.   0.02 0.05 0.75 0.17 0.01 0.01]
 [0.   0.   0.   0.   0.   0.05 0.78 0.16 0.01]
 [0.   0.   0.   0.   0.   0.02 0.   0.85 0.13]
 [0.   0.   0.   0.   0.   0.   0.   0.   1.  ]]
